# Junções

Existem diversas variedades da operação de junção. Porém, de longe a mais importante é a chamada junção natural, tanto que, o termo junção não qualificado é empregado invariavelmente para indicar de forma específica a junção natural.

#### Relações

Para os exemplos a seguir, considere as seguintes relações

`S` - Tropas

|cod|divisão|
|---|-------|
|1|Polícia Militar|
|2|Guarnição|
|3|Reconhecimento|
|4|Esquadrão de Cadetes|

`R` - Membros

|cod|tropa|nome|
|---|-----|----|
|1|2|Kitz Weilman|
|2|3|Levi Ackerman|
|3|2|Rico Brzenska|
|4|-1|Annie Leonhart|
|5|3|Jean Kirstein|
|6|3|Armin Arlert|
|7|-1|Reiner Braun|
|8|1|Hitch Dreyse|

In [1]:
# Declaração das Relações
from csvms.table import Table

tropas = Table(
    name="tropas",
    columns={
        "cod": int,
        "divisão": str},
    data=[
        (1,'Polícia Militar'),
        (2,'Guarnição'),
        (3,'Reconhecimento'),
        (4,'Esquadrão de Cadetes'),
    ]
)
print(tropas)

membros = Table(
    name="membros",
    columns={
        "cod": int,
        "tropa": int,
        "nome": str},
    data=[
        (1,2,'Kitz Weilman'),
        (2,3,'Levi Ackerman'),
        (3,2,'Rico Brzenska'),
        (4,-1,'Annie Leonhart'),
        (5,3,'Jean Kirstein'),
        (6,3,'Armin Arlert'),
        (7,-1,'Reiner Braun'),
        (8,1,'Hitch Dreyse'),
    ]
)
print(membros)

TABLE: default.tropas
   +----+--------------------+
   |cod |divisão             |
   +----+--------------------+
  0|   1|     Polícia Militar|
  1|   2|           Guarnição|
  2|   3|      Reconhecimento|
  3|   4|Esquadrão de Cadetes|
   +----+--------------------+

TABLE: default.membros
   +----+------+--------------+
   |cod |tropa |nome          |
   +----+------+--------------+
  0|   1|     2|  Kitz Weilman|
  1|   2|     3| Levi Ackerman|
  2|   3|     2| Rico Brzenska|
  3|   4|    -1|Annie Leonhart|
  4|   5|     3| Jean Kirstein|
  5|   6|     3|  Armin Arlert|
  6|   7|    -1|  Reiner Braun|
  7|   8|     1|  Hitch Dreyse|
   +----+------+--------------+



## Natural

O resultado da junção natural é uma tabela com todas as combinações das tuplas em R e S que seu atributos em comum são iguais.

### `×` Cruzada

```sql
SELECT * FROM membros R, tropas S WHERE R.tropa = S.cod
```

In [2]:
# cross join
cross_cols = dict()
cross_cols.update({f"membros.{k}":v for k,v in membros.columns.items()})
cross_cols.update({f"tropas.{k}":v for k,v in tropas.columns.items()})
cross_rows = list()

for i, membro in enumerate(membros):
    for j, tropa in enumerate(tropas):
        if membros[i]['tropa']==tropas[j]['cod']:
            cross_rows.append(membro+tropa)

print(
    Table(
        name="tmp.cross",
        columns=cross_cols,
        data=cross_rows
    ).show(size=100,trunc=False)
)

TABLE: tmp.cross
   +------------+--------------+-------------+-----------+---------------+
   |membros.cod |membros.tropa |membros.nome |tropas.cod |tropas.divisão |
   +------------+--------------+-------------+-----------+---------------+
  0|           1|             2| Kitz Weilman|          2|      Guarnição|
  1|           2|             3|Levi Ackerman|          3| Reconhecimento|
  2|           3|             2|Rico Brzenska|          2|      Guarnição|
  3|           5|             3|Jean Kirstein|          3| Reconhecimento|
  4|           6|             3| Armin Arlert|          3| Reconhecimento|
  5|           8|             1| Hitch Dreyse|          1|Polícia Militar|
   +------------+--------------+-------------+-----------+---------------+



### `⋈` Interna

```sql
SELECT * FROM membros R JOIN tropas S ON R.tropa = S.cod
```

In [3]:
# inner join
inner_cols = dict()
inner_cols.update({f"membros.{k}":v for k, v in membros.columns.items()})
inner_cols.update({f"tropas.{k}":v for k, v in tropas.columns.items()})
inner_rows = list()

for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]}):
        inner_rows.append(membro + tropa)

print(
    Table(
        name="tmp.inner",
        columns=inner_cols,
        data=inner_rows
    ).show(size=100,trunc=False)
)

TABLE: tmp.inner
   +------------+--------------+-------------+-----------+---------------+
   |membros.cod |membros.tropa |membros.nome |tropas.cod |tropas.divisão |
   +------------+--------------+-------------+-----------+---------------+
  0|           1|             2| Kitz Weilman|          2|      Guarnição|
  1|           2|             3|Levi Ackerman|          3| Reconhecimento|
  2|           3|             2|Rico Brzenska|          2|      Guarnição|
  3|           5|             3|Jean Kirstein|          3| Reconhecimento|
  4|           6|             3| Armin Arlert|          3| Reconhecimento|
  5|           8|             1| Hitch Dreyse|          1|Polícia Militar|
   +------------+--------------+-------------+-----------+---------------+



## Externa

O resultado do junção externa é o conjunto de todas as combinações das tuplas em R e S que seus atributos em comum são iguais , além disso, tuplas em R que não tem correspondência em S.

### `⟕` Esquerda

```sql
SELECT * FROM membros R LEFT JOIN tropas S ON R.tropa = S.cod
```

In [ ]:
# left outer join
left_cols = dict()
left_cols.update({f"membros.{k}":v for k, v in membros.columns.items()})
left_cols.update({f"tropas.{k}":v for k, v in tropas.columns.items()})
left_rows = list()
for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]},null=True):
        left_rows.append(membro + tropa)
print(
    Table(
        name="tmp.left",
        columns=left_cols,
        data=left_rows
    ).show(trunc=False)
)


In [11]:
# left with coalesce
left_cols = dict()
left_cols.update({f"membros.{k}":v for k, v in membros.columns.items()})
left_cols.update({f"tropas.{k}":v for k, v in tropas.columns.items()})
left_rows = list()
for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]},null=True):
        left_rows.append(membro + tropa)
print(
    Table(
        name="tmp.left",
        columns=left_cols,
        data=left_rows)
    .Π({'coalesce':['tropas.divisão','desempregado']},'divisao')
    .π([{'value':'membros.nome'},{'value':'divisao'}])
)


TABLE: default.((leftΠ)π)
   +--------------+---------------+
   |nome          |divisao        |
   +--------------+---------------+
  0|  Kitz Weilman|      Guarnição|
  1| Levi Ackerman| Reconhecimento|
  2| Rico Brzenska|      Guarnição|
  3|Annie Leonhart|   desempregado|
  4| Jean Kirstein| Reconhecimento|
  5|  Armin Arlert| Reconhecimento|
  6|  Reiner Braun|   desempregado|
  7|  Hitch Dreyse|Polícia Militar|
   +--------------+---------------+



### `⟖` Direita

```sql
SELECT * FROM membros R RIGHT JOIN tropas S ON R.tropa = S.cod
```

In [12]:
# right outer join
right_cols = dict()
right_cols.update({f"membros.{k}":v for k, v in membros.columns.items()})
right_cols.update({f"tropas.{k}":v for k, v in tropas.columns.items()})
right_rows = list()
for tropa in tropas:
    for membro in membros.σ({'eq':['tropa',tropa[0]]}, null=True):
        right_rows.append(membro + tropa)
print(
    Table(
        name="tmp.right",
        columns=right_cols,
        data=right_rows
    ).show(trunc=False)
)

TABLE: tmp.right
   +------------+--------------+-------------+-----------+--------------------+
   |membros.cod |membros.tropa |membros.nome |tropas.cod |tropas.divisão      |
   +------------+--------------+-------------+-----------+--------------------+
  0|           8|             1| Hitch Dreyse|          1|     Polícia Militar|
  1|           1|             2| Kitz Weilman|          2|           Guarnição|
  2|           3|             2|Rico Brzenska|          2|           Guarnição|
  3|           2|             3|Levi Ackerman|          3|      Reconhecimento|
  4|           5|             3|Jean Kirstein|          3|      Reconhecimento|
  5|           6|             3| Armin Arlert|          3|      Reconhecimento|
  6|        None|          None|         None|          4|Esquadrão de Cadetes|
   +------------+--------------+-------------+-----------+--------------------+



### `⟗` Completa

```sql
SELECT * FROM membros R FULL OUTER JOIN tropas S ON R.tropa = S.cod
```

In [13]:
# full outer join
full_cols = dict()
full_cols.update({f"membros.{k}":v for k, v in membros.columns.items()})
full_cols.update({f"tropas.{k}":v for k, v in tropas.columns.items()})

full_rows = list()

for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]},null=True):
        full_rows.append(membro + tropa)
for tropa in tropas:
    for membro in membros.σ({'eq':['tropa',tropa[0]]},null=True):
        full_rows.append(membro + tropa)
print(
    Table(
        name="tmp.full",
        columns=full_cols,
        # Remove duplicates
        data=list(dict.fromkeys(full_rows))
    ).show(trunc=False)
)


TABLE: tmp.full
   +------------+--------------+--------------+-----------+--------------------+
   |membros.cod |membros.tropa |membros.nome  |tropas.cod |tropas.divisão      |
   +------------+--------------+--------------+-----------+--------------------+
  0|           1|             2|  Kitz Weilman|          2|           Guarnição|
  1|           2|             3| Levi Ackerman|          3|      Reconhecimento|
  2|           3|             2| Rico Brzenska|          2|           Guarnição|
  3|           4|            -1|Annie Leonhart|       None|                None|
  4|           5|             3| Jean Kirstein|          3|      Reconhecimento|
  5|           6|             3|  Armin Arlert|          3|      Reconhecimento|
  6|           7|            -1|  Reiner Braun|       None|                None|
  7|           8|             1|  Hitch Dreyse|          1|     Polícia Militar|
  8|        None|          None|          None|          4|Esquadrão de Cadetes|
   +--------

## Semijunção
O resultado da semijunção é o conjunto de todas as tuplas em R para o qual existe uma tupla em S que é igual em seus nomes de atributos comuns.

### `⋉` Esquerda

```sql
SELECT * FROM membros R LEFT SEMI JOIN tropas S ON R.tropa = S.cod
```

In [14]:
# semi left join
semi_left_rows = list()
for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]}):
        semi_left_rows.append(membro)
print(
    Table(
        name="tmp.semi_left",
        columns=membros.columns,
        data=semi_left_rows
    ).show(trunc=False)
)

TABLE: tmp.semi_left
   +----+------+-------------+
   |cod |tropa |nome         |
   +----+------+-------------+
  0|   1|     2| Kitz Weilman|
  1|   2|     3|Levi Ackerman|
  2|   3|     2|Rico Brzenska|
  3|   5|     3|Jean Kirstein|
  4|   6|     3| Armin Arlert|
  5|   8|     1| Hitch Dreyse|
   +----+------+-------------+



### `⋊` Direita

```sql
SELECT * FROM membros R RIGHT SEMI JOIN JOIN tropas S ON R.tropa = S.cod
```

In [15]:
# semi right join
semi_right_rows = list()
for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]}):
        semi_right_rows.append(tropa)
print(
    Table(
        name="tmp.semi_left",
        columns=tropas.columns,
        data=semi_right_rows
    ).show(trunc=False)
)

TABLE: tmp.semi_left
   +----+---------------+
   |cod |divisão        |
   +----+---------------+
  0|   2|      Guarnição|
  1|   3| Reconhecimento|
  2|   2|      Guarnição|
  3|   3| Reconhecimento|
  4|   3| Reconhecimento|
  5|   1|Polícia Militar|
   +----+---------------+



## Antijunção

O resultado de uma antijunção é apenas aquelas tuplas em R para as quais NÃO existe uma tupla em S que possua os mesmos nomes de atributos.

### `▷` Esquerda

```sql
SELECT * FROM membros R LEFT ANTI JOIN tropas S ON R.tropa = S.cod
```

In [16]:
# anti left join
left_anti_rows = list()
for membro in membros:
    for tropa in tropas.σ({'eq':['cod',membro[1]]},null=True):
        if tropa[0] is None:
            left_anti_rows.append(membro)
print(
    Table(
        name="tmp.left_anti",
        columns=membros.columns,
        data=left_anti_rows
    ).show(trunc=False)
)


TABLE: tmp.left_anti
   +----+------+--------------+
   |cod |tropa |nome          |
   +----+------+--------------+
  0|   4|    -1|Annie Leonhart|
  1|   7|    -1|  Reiner Braun|
   +----+------+--------------+



### `◁` Direita

```sql
SELECT * FROM tropas S RIGHT ANTI JOIN membros R ON R.tropa = S.cod
```

In [17]:
# anti right join
anti_right_rows = list()
for tropa in tropas:
    for membro in membros.σ({'eq':['tropa',tropa[0]]}, null=True):
        if membro[1] is None:
            anti_right_rows.append(tropa)
print(
    Table(
        name="tmp.anti_right",
        columns=tropas.columns,
        data=anti_right_rows
    ).show(trunc=False)
)


TABLE: tmp.anti_right
   +----+--------------------+
   |cod |divisão             |
   +----+--------------------+
  0|   4|Esquadrão de Cadetes|
   +----+--------------------+

